# Imports and Global Variables

In [1]:
from typing import List
from config.endpoints import MAINNET_BASE_URL
from helpers.utility import _make_get_request, _make_post_request
from helius import NFTAPI, NameAPI, BalancesAPI, WebhooksAPI, TransactionsAPI
from dotenv import load_dotenv
from pathlib import Path
import requests
import pandas as pd
import numpy as np
import os
import time
import json
from collections import defaultdict


In [2]:
try:
    ROOT_DIR = Path(__file__).resolve().parent
except NameError:
    ROOT_DIR = Path(os.getcwd()).resolve()

In [3]:
ROOT_DIR

WindowsPath('C:/Users/Brandyn/projects/solana_analysis_tool/notebooks')

In [4]:
# Persistent Variables

LAMPORT_SCALE = 1e9
WRAPPED_SOL = "So11111111111111111111111111111111111111112"
NATIVE_SOL  = "So11111111111111111111111111111111111111111"

In [5]:
pd.set_option('display.max_colwidth', None)  
pd.set_option('display.max_columns', None)   
pd.set_option('display.width', 200)    
pd.set_option('display.float_format', '{:.8f}'.format)

In [6]:
load_dotenv()

True

In [7]:
HELIUS_API_KEY = os.getenv('HELIUS_API_KEY')
VYBE_API_KEY = os.getenv('VYBE_API_KEY')
NANSEN_API_KEY = os.getenv('NANSEN_API_KEY')


In [8]:
use_cache = True
test_address = 'AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU'

# Flipside Data (Balances over time)

Here is the Flipside Query; when moving to live app will use the Flipside API to retrieve this dynamically: https://flipsidecrypto.xyz/studio/queries/f72285d7-f3d4-4596-a9cd-8785187005a2

In [9]:
os.getcwd()

'c:\\Users\\Brandyn\\projects\\solana_analysis_tool\\notebooks'

In [10]:
balance_df = pd.read_csv('../data/raw/solana_balance_history_test.csv').dropna(how='all')

# Strip BOMs or invisible characters from column names
balance_df.columns = balance_df.columns.str.replace('\ufeff', '', regex=False).str.strip()

# Optionally: remove rows where any column has just a BOM or is empty/whitespace
balance_df = balance_df[~balance_df.apply(lambda row: row.astype(str).str.contains('\ufeff|^\s*$', regex=True)).any(axis=1)]

sol_mask = (
    (balance_df['MINT'] == 'So11111111111111111111111111111111111111111') &
    (balance_df['SYMBOL'].isna())
)

balance_df.loc[sol_mask, 'SYMBOL'] = 'SOL'
balance_df.loc[sol_mask, 'NAME'] = 'Solana'

# balance_df['TX_ID'] = balance_df['TX_ID'].str.lower()

<>:7: SyntaxWarning: invalid escape sequence '\s'
<>:7: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Brandyn\AppData\Local\Temp\ipykernel_11744\1471790978.py:7: SyntaxWarning: invalid escape sequence '\s'
  balance_df = balance_df[~balance_df.apply(lambda row: row.astype(str).str.contains('\ufeff|^\s*$', regex=True)).any(axis=1)]


In [11]:
def collapse_token_balances(group):
    return group.apply(lambda row: {
        "mint": row["MINT"],
        "symbol": row["SYMBOL"],
        "name": row["NAME"],
        "pre_balance": row["PRE_BALANCE"],
        "post_balance": row["BALANCE"],
    }, axis=1).tolist()


In [12]:
# Any nan values left should be LP tokens

balance_df[balance_df['SYMBOL'].isna()]

,BLOCK_TIMESTAMP,OWNER,MINT,PRE_BALANCE,BALANCE,TX_ID,SUCCEEDED,SYMBOL,NAME
490,2024-01-27 20:02:00.000,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,9HB4kAMLSYfGFfN142DKMyPyHyZQ8pXF8M1STbDudodY,21.28450600,0.00000000,AG9Wy4nnG7vgMgVwmBmiKUdfcGk9Sigh9XHBgUMq9LCfL8PuMB6vUzv16yCBS4BMPHhdL9coGEq6GyBAT7gKbw6,True,NaN,NaN
494,2024-01-27 20:00:42.000,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,9HB4kAMLSYfGFfN142DKMyPyHyZQ8pXF8M1STbDudodY,0.00000000,21.28450600,3kTQ7DYxvUoknsnHkHd4wMGCvXgq5CdXM6ZPUUnfJc1tGBYoEBKtArBUTumUCzgocYbR82598vr8jzVebKe5ekiF,True,NaN,NaN
503,2024-01-27 19:56:38.000,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,9HB4kAMLSYfGFfN142DKMyPyHyZQ8pXF8M1STbDudodY,38.26138600,0.00000000,5XbLMj697ixEAzt6aZMD3fXx26cKYUwHmA2PmqMPYAcDryNqpEsG7Q3cmKEW5tpnkXSAVFb9A3ihrQV71fDSX9aq,True,NaN,NaN
510,2024-01-27 19:54:37.000,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,9HB4kAMLSYfGFfN142DKMyPyHyZQ8pXF8M1STbDudodY,0.00000000,38.26138600,2JGMFWYbCcR51SgXUPejvvpRRHAs1xyxPgTz96DrrXNNmbZHdeF9ASVa84PPiw7s8Y6N1Jy41CH8rJjM26sxjutb,True,NaN,NaN
519,2024-01-27 19:52:56.000,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,9HB4kAMLSYfGFfN142DKMyPyHyZQ8pXF8M1STbDudodY,12.88031600,0.00000000,WMkk8qPz8xNk1kw1zaggErsBBbegDM6PRXWYiDvbDcU4WvNZ2o5hQ2UFfbpFt4oj333SsHsFNFmW4X4SFJkfKKs,True,NaN,NaN
522,2024-01-27 19:51:51.000,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,9HB4kAMLSYfGFfN142DKMyPyHyZQ8pXF8M1STbDudodY,10.00000000,12.88031600,t1EgxX8diQh5Ak4ptKnY6ajRu96dwE2R8yYJraQsSJ4CxyLSjKJ2qevrgeceJqYEnq9wob6baGkv4GpThh551UB,True,NaN,NaN
524,2024-01-27 19:47:10.000,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,9HB4kAMLSYfGFfN142DKMyPyHyZQ8pXF8M1STbDudodY,0.00000000,10.00000000,DEYMhgGRURmjVATdnhMQAUsZFxHzRXftxPS3E5BGteS7VJZXgwuJW52SSRbHgV5cLy1b9NTx1Au32KBv9DRzune,True,NaN,NaN
539,2024-01-12 00:29:26.000,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,B9uEZfxJoAsc51BZs6rNs8XtYVu5vfVrhbCuDCJ7mPEE,0.00000000,5.00000000,qWSnf1ioo2UCggYWbZDFa1KnAv8VUJgg7dKWGKAZFYP6q6Adkq3LV2vbiCazS8AFn8Ro1dSAqZGgwyaxq5mdAe5,True,NaN,NaN
540,2024-01-11 23:02:04.000,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,9HB4kAMLSYfGFfN142DKMyPyHyZQ8pXF8M1STbDudodY,5.71174900,0.00000000,WNZ5j6dcGtjmigf8dXVbH72T4ks6Bo3kvtSgMBzL7trJdNZtpvJo2BFctkfXwDhBtwNdxVBvQYcvimoM61DXAYB,True,NaN,NaN
544,2024-01-11 22:57:58.000,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,9HB4kAMLSYfGFfN142DKMyPyHyZQ8pXF8M1STbDudodY,0.00000000,5.71174900,5238f2M2JwJBSqSNb5k1L8B6mj5Z9D1XS9EEaKWzjKpZSf9KZpSCkTTxgrDEic1peFKhuib5VimcWbPNendqUzef,True,NaN,NaN


# Helius API Calls

In [13]:
def get_all_signatures(account_address, helius_api_key, max_pages=20, limit=100):
    url = f"https://mainnet.helius-rpc.com/?api-key={helius_api_key}"
    collected = []
    before = None

    for _ in range(max_pages):
        payload = {
            "jsonrpc": "2.0",
            "id": "1",
            "method": "getSignaturesForAddress",
            "params": [account_address, {"limit": limit, **({"before": before} if before else {})}]
        }

        response = requests.post(
            url,
            headers={"Content-Type": "application/json"},
            data=json.dumps(payload)
        )

        try:
            data = response.json()
            if "error" in data:
                print(f"❌ Error: {data['error']['message']}")
                break

            batch = data["result"]
            if not batch:
                break

            collected.extend(batch)
            before = batch[-1]["signature"]  # Move to next batch

            time.sleep(0.5)  # Avoid rate-limits

        except Exception as e:
            print(f"Exception: {e}")
            break

    return collected

In [14]:
# signatures = get_all_signatures(test_address, HELIUS_API_KEY)

# signatures_dict = {}

# for sig in signatures:
#     time = sig.get('blockTime')
#     signature_str = sig.get('signature')

#     signatures_dict[signature_str] = time

# signatures_dict

In [15]:
# signatures_array = list(signatures_dict.keys())

In [16]:
def v0_transactions_all(signatures, helius_api_key):
    import time
    url = f"https://api.helius.xyz/v0/transactions?api-key={helius_api_key}"
    headers = {"Content-Type": "application/json"}

    all_results = []
    batch_size = 100

    for i in range(0, len(signatures), batch_size):
        batch = signatures[i:i+batch_size]
        payload = json.dumps({"transactions": batch})

        response = requests.post(url, headers=headers, data=payload)

        try:
            data = response.json()
            if isinstance(data, dict) and "error" in data:
                print(f"❌ Error at batch {i // batch_size}: {data['error']}")
                continue

            all_results.extend(data)

        except Exception as e:
            print(f"❌ Exception during batch {i // batch_size}: {e}")

        time.sleep(0.3)  # Optional rate limit buffer

    return all_results


In [17]:
# data = v0_transactions_all(signatures_array, HELIUS_API_KEY)
# len(data)

In [18]:
def get_comprehensive_tx_history(wallet, api_key, use_cache=True):

    if use_cache:
        with open('../data/raw/test_tx_history.json') as f:
            parsed_transaction_history = json.load(f)

        return parsed_transaction_history
    
    else:

        signatures = get_all_signatures(wallet, api_key)

        signatures_dict = {}

        for sig in signatures:
            time = sig.get('blockTime')
            signature_str = sig.get('signature')

            signatures_dict[signature_str] = time
        
        signatures_array = list(signatures_dict.keys())

        parsed_transaction_history = v0_transactions_all(signatures_array, api_key)

        with open('../data/raw/test_tx_history.json', 'w') as f:
            json.dump(parsed_transaction_history, f)

        return parsed_transaction_history

In [19]:
parsed_transaction_history = get_comprehensive_tx_history(test_address, HELIUS_API_KEY, use_cache=use_cache)
len(parsed_transaction_history)

493

In [20]:
def summarize_transaction(tx, wallet):
    base = {
        "timestamp": tx.get("timestamp"),
        "signature": tx.get("signature"),
        "type": tx.get("type", "UNKNOWN"),
        "source": tx.get("source", "UNKNOWN"),
        "tx_status": int(tx.get("transactionError") is None),
        "block_number": tx.get("slot"),
    }

    # Track counterparties
    rows = []
    counterparties = set()

    # Native transfers
    seen_native = set()
    for t in tx.get("nativeTransfers", []):
        amount = t.get("amount", 0) / LAMPORT_SCALE
        from_user = t.get("fromUserAccount")
        to_user = t.get("toUserAccount")
        
        transfer_id = (from_user, to_user, amount)
        if transfer_id in seen_native:
            continue
        seen_native.add(transfer_id)

        if wallet in [from_user, to_user]:
            row = base.copy()
            row.update({
                "token_address": NATIVE_SOL,
                # "symbol": "SOL",
                "token_amount": amount,
                "direction": "sent" if from_user == wallet else "received",
                "sender": from_user,
                "receiver": to_user,
                "counterparty": to_user if from_user == wallet else from_user,
            })
            rows.append(row)


    # Token transfers
    for t in tx.get("tokenTransfers", []):
        try:
            amount = float(t.get("tokenAmount", 0))
        except (ValueError, TypeError):
            amount = 0.0

        mint = t.get("mint", "UNKNOWN")
        from_user = t.get("fromUserAccount")
        to_user = t.get("toUserAccount")

        row = base.copy()
        row.update({
            "token_address": mint,
            "token_amount": amount,
            "direction": "sent" if from_user == wallet else "received",
            "sender": from_user,
            "receiver": to_user,
            "counterparty": to_user if from_user == wallet else from_user,
        })
        if wallet in [from_user, to_user]:
            counterparties.add(row["counterparty"])
            rows.append(row)
    if not rows:
        for acc in tx.get("accountData", []):
            acct = acc.get("account")
            native_change = acc.get("nativeBalanceChange", 0) / LAMPORT_SCALE
            if acct == wallet:
                row = base.copy()
                row.update({
                    "token_address": NATIVE_SOL,
                    "token_amount": abs(native_change),
                    "direction": "received" if native_change > 0 else "sent",
                    "sender": None,
                    "receiver": wallet,
                    "counterparty": None,
                })
                rows.append(row)

    for event in tx.get("events", {}).get("compressed", []):
        if event.get("type") == "COMPRESSED_NFT_MINT":
            if event.get("newLeafOwner") == wallet:
                row = base.copy()
                row.update({
                    "token_address": "COMPRESSED_NFT",
                    "token_amount": 1,
                    "direction": "received",
                    "sender": event.get("treeDelegate"),
                    "receiver": wallet,
                    "counterparty": event.get("treeDelegate"),
                    "symbol": event.get("metadata", {}).get("name", "NFT"),
                })
                rows.append(row)

    return rows


In [21]:
tx_summary = summarize_transaction(parsed_transaction_history[50], test_address)
pd.DataFrame(tx_summary)

,timestamp,signature,type,source,tx_status,block_number,token_address,token_amount,direction,sender,receiver,counterparty
0,1740546718,kg5bLy67R6JuSkjiFCS9mByEx9VetbeMNckxHJr3XsbdB5g4HjU9B7RcuhLapkLU9vU9C1QieNGnFzyLoTSMLLQ,UNKNOWN,JUPITER,1,323149064,Es9vMFrzaCERmJfrF4H2FYD4KCoNkY11McCe8BenwNYB,394.20629100,sent,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,GGztQqQ6pCPaJQnNpXBgELr5cs3WwDakRbh1iEMzjgSJ,GGztQqQ6pCPaJQnNpXBgELr5cs3WwDakRbh1iEMzjgSJ
1,1740546718,kg5bLy67R6JuSkjiFCS9mByEx9VetbeMNckxHJr3XsbdB5g4HjU9B7RcuhLapkLU9vU9C1QieNGnFzyLoTSMLLQ,UNKNOWN,JUPITER,1,323149064,2b1kV6DkPAnxd5ixfnxCpjxmKwqjjaYmCZfHsFu24GXo,393.75022700,received,9tXiuRRw7kbejLhZXtxDxYs2REe43uH2e7k1kocgdM9B,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,9tXiuRRw7kbejLhZXtxDxYs2REe43uH2e7k1kocgdM9B
2,1740546718,kg5bLy67R6JuSkjiFCS9mByEx9VetbeMNckxHJr3XsbdB5g4HjU9B7RcuhLapkLU9vU9C1QieNGnFzyLoTSMLLQ,UNKNOWN,JUPITER,1,323149064,2b1kV6DkPAnxd5ixfnxCpjxmKwqjjaYmCZfHsFu24GXo,3.34687600,sent,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,8psNvWTrdNTiVRNzAgsou9kETXNJm2SXZyaKuJraVRtf,8psNvWTrdNTiVRNzAgsou9kETXNJm2SXZyaKuJraVRtf


In [22]:
def get_instruction_data(tx, wallet):
    instructions_data = {}

    for i in tx.get('instructions'):
        accounts = i['accounts']
        if accounts and wallet in accounts:
            data = i['data']
            program_id = i['programId']
            instructions_data['data'] = data
            instructions_data['programId'] = program_id

    return instructions_data

## Transaction-Level Dataset

In [23]:
#Helper function to create row from helius and flipside data

def create_row(tx, wallet, balance_df):
    """
    Takes in a tx from helius data, a wallet address, and balance timeseries data from Flipside.  Returns a df row for analysis 
    """
    tx_summary = summarize_transaction(tx, wallet)
    instructions_data = get_instruction_data(tx, wallet)

    # print(type(tx_summary))

    token_tx_df = pd.DataFrame(tx_summary)

    tx_status = "failed" if tx.get("transactionError") else "success"

    token_tx_df['block_number'] = tx.get('slot')
    token_tx_df['tx_fee'] = tx.get('fee') / LAMPORT_SCALE
    token_tx_df['program_id'] = instructions_data.get('programId')
    token_tx_df['tx_status'] = tx_status #if no tx error, can assume it succeeded so we use binary 1 and 0 instead of just leaving nan
    # token_tx_df['signature'] = token_tx_df['signature'].str.lower()

    filtered_balance_timeseries = balance_df[['PRE_BALANCE','BALANCE','SYMBOL','NAME','MINT','TX_ID']].rename(columns={'MINT':'token_address'})
    filtered_balance_timeseries = filtered_balance_timeseries.rename(columns={'TX_ID':'signature'})
    # print(f'token_tx_df: {token_tx_df}')
    # print(f'filtered_balance_timeseries: {filtered_balance_timeseries}')

    if token_tx_df.empty:
        print(f'tx: {tx}')
        print(f'tx_summary: {tx_summary}')

    # import pdb; pdb.set_trace()

    if not token_tx_df.empty:
        combined_df = pd.merge(token_tx_df, filtered_balance_timeseries, on=['signature','token_address'], how='left')
    else:
        combined_df = pd.DataFrame()
        
    return combined_df 



In [24]:
#Testing the workflow to concat the dataset

combined_row = create_row(parsed_transaction_history[0], test_address, balance_df)
combined_row2 = create_row(parsed_transaction_history[1], test_address, balance_df)
pd.concat([combined_row,combined_row2]).drop_duplicates(subset='signature')

,timestamp,signature,type,source,tx_status,block_number,token_address,token_amount,direction,sender,receiver,counterparty,tx_fee,program_id,PRE_BALANCE,BALANCE,SYMBOL,NAME
0,1743941539,6mPqrKS4AKYJeZ3JLjjubHSwNB96oiJdFX1DgeZNZEBHxMTEx9RUZYt2SyFRvpPohpXm9RRABbVb2CEHgpdTzGv,TRANSFER,SYSTEM_PROGRAM,success,331670303,So11111111111111111111111111111111111111111,0.00000000,received,5Hr7wZg7oBpVhH5nngRqzr5W7ZFUfCsfEhbziZJak7fr,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,5Hr7wZg7oBpVhH5nngRqzr5W7ZFUfCsfEhbziZJak7fr,0.00000583,11111111111111111111111111111111,0.01777252,0.01777252,SOL,Solana
0,1743898250,29aP1yrCx2dbJM4Mxk8257G5e87bY7CgxySDS6LiAAmsxXLspNqdbGpYZFYiwJqjPLwBU5V6gwZ2gvuf4FSc5YrU,TRANSFER,SYSTEM_PROGRAM,success,331561835,So11111111111111111111111111111111111111111,0.00000000,received,FLiPgGTXtBtEJoytikaywvWgbz5a56DdHKZU72HSYMFF,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,FLiPgGTXtBtEJoytikaywvWgbz5a56DdHKZU72HSYMFF,0.00000500,11111111111111111111111111111111,0.01777252,0.01777252,SOL,Solana


In [25]:
for a in parsed_transaction_history[0]['accountData']:
    if a.get('account') == test_address:
        print(f'a: {a}')

a: {'account': 'AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU', 'nativeBalanceChange': 1, 'tokenBalanceChanges': []}


In [26]:
len(parsed_transaction_history)

493

In [27]:
tx_level_data = pd.DataFrame()

for i in parsed_transaction_history:
    row = create_row(i, test_address, balance_df)
    
    tx_level_data = pd.concat([tx_level_data, row])


In [28]:
tx_level_data['signature'].nunique()

493

In [29]:
tx_level_data[tx_level_data['token_address']=='COMPRESSED_NFT']

,timestamp,signature,type,source,tx_status,block_number,token_address,token_amount,direction,sender,receiver,counterparty,tx_fee,program_id,PRE_BALANCE,BALANCE,SYMBOL,NAME,symbol
1,1738604617,3TPZSpTA5QKz5AQ3g7yX2ELnQHWYRGiuKu9T8GgVjBfSvZGgjxfi7eVporwQGaEWzw1JhM56Nn8sk3ijSPFd8nML,COMPRESSED_NFT_MINT,BUBBLEGUM,success,318251172,COMPRESSED_NFT,1.00000000,received,GwxDwNvmh5exDPJibZL2ztiecW7aSYoa2rca3zvN1MRB,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,GwxDwNvmh5exDPJibZL2ztiecW7aSYoa2rca3zvN1MRB,0.00000500,BGUMAp9Gq7iTEuizy4pqaxsTyUCBK68MDfK752saRPUY,NaN,NaN,NaN,NaN,AB Open #001
1,1738604602,35ie9YJLNVcMGA751C2aN2JHoN7YidMmgntEw2pg3QLEzwM1qKAwdDfUxfkrjvCaDFCeqECUxnopH6v3K6SYC7Cm,COMPRESSED_NFT_MINT,BUBBLEGUM,success,318251136,COMPRESSED_NFT,1.00000000,received,GwxDwNvmh5exDPJibZL2ztiecW7aSYoa2rca3zvN1MRB,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,GwxDwNvmh5exDPJibZL2ztiecW7aSYoa2rca3zvN1MRB,0.00000500,BGUMAp9Gq7iTEuizy4pqaxsTyUCBK68MDfK752saRPUY,NaN,NaN,NaN,NaN,AB Open #001
1,1735608468,5PKmHyHf9C6ASTt2HWqXy3iMuRLofE3i6HAB88kYEHUuFCRKhyDq5gyyotq3yCxvs1xZRP9CF1C8CJDQ7yPYd7yv,COMPRESSED_NFT_MINT,BUBBLEGUM,success,310883171,COMPRESSED_NFT,1.00000000,received,59suVJh2jLvADqgxv3cUmNMm9g3hjXb5M6vu29jV27Tt,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,59suVJh2jLvADqgxv3cUmNMm9g3hjXb5M6vu29jV27Tt,0.00000632,BGUMAp9Gq7iTEuizy4pqaxsTyUCBK68MDfK752saRPUY,NaN,NaN,NaN,NaN,Grant #3
1,1735579228,5zMbFS9efMXYjwN28zgzDGo5j8VNiPikgc8YAv1EfHUBtX1NgsvoUjduMN5fjw89wt6YHZt6QmZcW7Weqj5b3GGU,COMPRESSED_NFT_MINT,BUBBLEGUM,success,310811606,COMPRESSED_NFT,1.00000000,received,CaUju6XDMoLWpd5if1es3FQWFR71tzGMQ9SsUSnceBYX,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,CaUju6XDMoLWpd5if1es3FQWFR71tzGMQ9SsUSnceBYX,0.00000500,BGUMAp9Gq7iTEuizy4pqaxsTyUCBK68MDfK752saRPUY,NaN,NaN,NaN,NaN,🌟 Gift #314
2,1735579228,5zMbFS9efMXYjwN28zgzDGo5j8VNiPikgc8YAv1EfHUBtX1NgsvoUjduMN5fjw89wt6YHZt6QmZcW7Weqj5b3GGU,COMPRESSED_NFT_MINT,BUBBLEGUM,success,310811606,COMPRESSED_NFT,1.00000000,received,CaUju6XDMoLWpd5if1es3FQWFR71tzGMQ9SsUSnceBYX,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,CaUju6XDMoLWpd5if1es3FQWFR71tzGMQ9SsUSnceBYX,0.00000500,BGUMAp9Gq7iTEuizy4pqaxsTyUCBK68MDfK752saRPUY,NaN,NaN,NaN,NaN,🌟 Gift #320
1,1735486501,2DVEywsq4Qt2KgkmZMuP3FrB1UZVPN6Rvi9rHUFYmoBEzzhrsQTRv3tyJaMt6AthVy7q1wdmrN6JDHnkqGhpBFHn,COMPRESSED_NFT_MINT,BUBBLEGUM,success,310585694,COMPRESSED_NFT,1.00000000,received,5JQ8b8ZHeu6iSKH8gH7Gee5jWPfW6hc4BZrW4FZpq3Kh,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,5JQ8b8ZHeu6iSKH8gH7Gee5jWPfW6hc4BZrW4FZpq3Kh,0.00000632,BGUMAp9Gq7iTEuizy4pqaxsTyUCBK68MDfK752saRPUY,NaN,NaN,NaN,NaN,☄️JUPITER☄️ AIRDROP
1,1735486471,4Fnj33Jq5xe7MZcdVa15su8VKMXXW6Jc2y6d4FDeoVEQj5v9ffQT8HQHPBxc5mKJWrV7q5vpDfM6vMariktDmE2x,COMPRESSED_NFT_MINT,BUBBLEGUM,success,310585620,COMPRESSED_NFT,1.00000000,received,5JQ8b8ZHeu6iSKH8gH7Gee5jWPfW6hc4BZrW4FZpq3Kh,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,5JQ8b8ZHeu6iSKH8gH7Gee5jWPfW6hc4BZrW4FZpq3Kh,0.00000632,BGUMAp9Gq7iTEuizy4pqaxsTyUCBK68MDfK752saRPUY,NaN,NaN,NaN,NaN,☄️JUPITER☄️ AIRDROP
1,1734970206,HB6rZmSiSNJoDTz45cwdmRDMCnJZKdLod2v9shroCBAdwfsWRZsP916CrQWqg6dpgeUeJsnouz7DZSq49M87uBN,COMPRESSED_NFT_MINT,BUBBLEGUM,success,309332138,COMPRESSED_NFT,1.00000000,received,4AuVuuzh7NA8b8jCcJzARdUALum4MxG9tPYA7QxTPHqo,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,4AuVuuzh7NA8b8jCcJzARdUALum4MxG9tPYA7QxTPHqo,0.00000730,BGUMAp9Gq7iTEuizy4pqaxsTyUCBK68MDfK752saRPUY,NaN,NaN,NaN,NaN,BOX#8
1,1734820722,4gtgKHj7aum7JBcb4Bm9Norw3216dQkzHKtxBFznskoB48FYGTUdmhnP7siG3z6juKwUN7dH1sj91Lypc2bq3unr,COMPRESSED_NFT_MINT,BUBBLEGUM,success,308967649,COMPRESSED_NFT,1.00000000,received,4AuVuuzh7NA8b8jCcJzARdUALum4MxG9tPYA7QxTPHqo,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,4AuVuuzh7NA8b8jCcJzARdUALum4MxG9tPYA7QxTPHqo,0.00000730,BGUMAp9Gq7iTEuizy4pqaxsTyUCBK68MDfK752saRPUY,NaN,NaN,NaN,NaN,5000 TO 50000 #1
1,1734820662,a8aKKJugCLBf1kC8brAakSkZAEHgztfiqgCth9ghNY33eHxfyDFxDE8GVXea4MZPAFnNa4P7DrLkfRSugzZio8r,COMPRESSED_NFT_MINT,BUBBLEGUM,success,308967503,COMPRESSED_NFT,1.000000

In [30]:
tx_level_data['timestamp'] = pd.to_datetime(tx_level_data['timestamp'], unit='s')
tx_level_data.rename(columns={'BALANCE':'POST_BALANCE','NAME':'TOKEN_NAME'},inplace=True)

In [31]:
tx_level_data[tx_level_data['signature']=='4oJDpv6EPm9zyh5ifuGUSkT6qe3mNDKHLWU7sbNBwPRY8CNTwU3SdoczL65axR1RfR1gAAuC9uUWY11BFTuWTWPW']

,timestamp,signature,type,source,tx_status,block_number,token_address,token_amount,direction,sender,receiver,counterparty,tx_fee,program_id,PRE_BALANCE,POST_BALANCE,SYMBOL,TOKEN_NAME,symbol
0,2025-04-05 23:35:46,4oJDpv6EPm9zyh5ifuGUSkT6qe3mNDKHLWU7sbNBwPRY8CNTwU3SdoczL65axR1RfR1gAAuC9uUWY11BFTuWTWPW,UNKNOWN,JUPITER,success,331556566,Es9vMFrzaCERmJfrF4H2FYD4KCoNkY11McCe8BenwNYB,129.49945900,sent,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,39GrsozbzM9Sg1U7EDnEtQ69fsVF3pmVtmV2DGDAQQJ5,39GrsozbzM9Sg1U7EDnEtQ69fsVF3pmVtmV2DGDAQQJ5,0.00008000,JUP6LkbZbjS1jKKwapdHNy74zcZ3tLUZoi5QNyVTaV4,129.49945900,0.00000000,USDT,Tether,NaN
1,2025-04-05 23:35:46,4oJDpv6EPm9zyh5ifuGUSkT6qe3mNDKHLWU7sbNBwPRY8CNTwU3SdoczL65axR1RfR1gAAuC9uUWY11BFTuWTWPW,UNKNOWN,JUPITER,success,331556566,2b1kV6DkPAnxd5ixfnxCpjxmKwqjjaYmCZfHsFu24GXo,129.41943700,received,39GrsozbzM9Sg1U7EDnEtQ69fsVF3pmVtmV2DGDAQQJ5,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,39GrsozbzM9Sg1U7EDnEtQ69fsVF3pmVtmV2DGDAQQJ5,0.00008000,JUP6LkbZbjS1jKKwapdHNy74zcZ3tLUZoi5QNyVTaV4,0.00000000,128.31937200,PYUSD,PayPal USD,NaN
2,2025-04-05 23:35:46,4oJDpv6EPm9zyh5ifuGUSkT6qe3mNDKHLWU7sbNBwPRY8CNTwU3SdoczL65axR1RfR1gAAuC9uUWY11BFTuWTWPW,UNKNOWN,JUPITER,success,331556566,2b1kV6DkPAnxd5ixfnxCpjxmKwqjjaYmCZfHsFu24GXo,1.10006500,sent,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,8psNvWTrdNTiVRNzAgsou9kETXNJm2SXZyaKuJraVRtf,8psNvWTrdNTiVRNzAgsou9kETXNJm2SXZyaKuJraVRtf,0.00008000,JUP6LkbZbjS1jKKwapdHNy74zcZ3tLUZoi5QNyVTaV4,0.00000000,128.31937200,PYUSD,PayPal USD,NaN


# Token Prices

In [32]:
tx_level_data['timestamp'].min()

Timestamp('2023-11-14 15:20:07')

In [33]:
tx_level_data.columns

Index(['timestamp', 'signature', 'type', 'source', 'tx_status', 'block_number', 'token_address', 'token_amount', 'direction', 'sender', 'receiver', 'counterparty', 'tx_fee', 'program_id',
       'PRE_BALANCE', 'POST_BALANCE', 'SYMBOL', 'TOKEN_NAME', 'symbol'],
      dtype='object')

In [34]:
tx_level_data[tx_level_data['token_address'].notnull()]['token_address']

0    So11111111111111111111111111111111111111111
0    So11111111111111111111111111111111111111111
0    So11111111111111111111111111111111111111111
0    So11111111111111111111111111111111111111111
0    So11111111111111111111111111111111111111111
                        ...                     
1    So11111111111111111111111111111111111111111
2    So11111111111111111111111111111111111111111
3    So11111111111111111111111111111111111111111
4    So11111111111111111111111111111111111111112
0    So11111111111111111111111111111111111111111
Name: token_address, Length: 985, dtype: object

In [35]:
# cleaned_tokens = tx_level_data[tx_level_data['token_address_sent'].notnull()]['token_address_sent'].unique()
# cleaned_tokens

We will use flipside query to get token prices for each token

https://flipsidecrypto.xyz/studio/queries/9207cf6d-d327-44cf-ab49-c8d6e48e7483

In [36]:
prices_data = pd.read_csv('../data/raw/test_address_prices.csv').dropna()
prices_data['DT'] = pd.to_datetime(pd.to_datetime(prices_data['DT']).dt.strftime('%Y-%m-%d'))
prices_data

,DT,SYMBOL,TOKEN_ADDRESS,PRICE
0,2025-04-18,BSOL,bSo13r4TkiE4KumL71LsHTPpL2euBYLFx6h9HP3piy1,162.26500000
1,2025-04-18,USDC,EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v,0.99994055
2,2025-04-18,PYUSD,2b1kV6DkPAnxd5ixfnxCpjxmKwqjjaYmCZfHsFu24GXo,0.99983073
3,2025-04-18,MSOL,mSoLzYCxHdYgdzU16g5QSh3i5K3z3KZK7ytfqcJm7So,172.44363636
4,2025-04-18,KMNO,KMNo3nJsBXfcpJTVhZcXLW7RmTwTt4GVFE7suUBo9sS,0.05032830
...,...,...,...,...
5628,2023-11-13,NEON,NeonTjSjsuo3rexg9o6vHuMXw62f9V7zvmu8M8Zut44,0.32698658
5629,2023-11-13,SOL,So11111111111111111111111111111111111111112,56.75380014
5630,2023-11-13,USDT,Es9vMFrzaCERmJfrF4H2FYD4KCoNkY11McCe8BenwNYB,1.00042550
5631,2023-11-13,BLZE,BLZEEuZUBVqFhj8adcCFPJvPVCiCyVmh3hkJMrU8KuJA,0.00048821


In [37]:
def to_time(df):
    time_cols = ['date','dt','hour','time','day','month','year','week','timestamp','date(utc)','block_timestamp']
    for col in df.columns:
        if col.lower() in time_cols and col.lower() != 'timestamp':
            df[col] = pd.to_datetime(df[col])
            df.set_index(col, inplace=True)
        elif col.lower() == 'timestamp':
            df[col] = pd.to_datetime(df[col], unit='ms')
            df.set_index(col, inplace=True)
    print(df.index)
    return df 

def clean_prices(prices_df):
    print('cleaning prices')
    # Pivot the dataframe
    prices_df = prices_df.drop_duplicates(subset=['dt', 'symbol'])
    prices_df_pivot = prices_df.pivot(
        index='dt',
        columns='symbol',
        values='price'
    )
    prices_df_pivot = prices_df_pivot.reset_index()

    # Rename the columns by combining 'symbol' with a suffix
    prices_df_pivot.columns = ['dt'] + [f'{col}_Price' for col in prices_df_pivot.columns[1:]]
    
    print(f'cleaned prices: {prices_df_pivot}')
    return prices_df_pivot

In [38]:
def data_cleaning(df,dropna=True,ffill=False):
    clean_df = clean_prices(df)
    clean_df = to_time(clean_df)
    if dropna == True:
        # clean_df = clean_df.dropna(axis=1, how='any')
        clean_df = clean_df.dropna()
    if ffill == True:
        clean_df = clean_df.resample('d').ffill().bfill()# filling to day

    if '__row_index' in clean_df.columns:
        clean_df.drop(columns=['__row_index'], inplace=True)

    return clean_df

In [39]:
prices_data.columns = prices_data.columns.str.lower()


In [40]:


# Duplicate wrapped SOL rows with native SOL address
wrapped_sol_prices = prices_data[prices_data['token_address'] == WRAPPED_SOL].copy()
wrapped_sol_prices['token_address'] = NATIVE_SOL

# Append to original price data
prices_data = pd.concat([prices_data, wrapped_sol_prices], ignore_index=True)
prices_data

,dt,symbol,token_address,price
0,2025-04-18,BSOL,bSo13r4TkiE4KumL71LsHTPpL2euBYLFx6h9HP3piy1,162.26500000
1,2025-04-18,USDC,EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v,0.99994055
2,2025-04-18,PYUSD,2b1kV6DkPAnxd5ixfnxCpjxmKwqjjaYmCZfHsFu24GXo,0.99983073
3,2025-04-18,MSOL,mSoLzYCxHdYgdzU16g5QSh3i5K3z3KZK7ytfqcJm7So,172.44363636
4,2025-04-18,KMNO,KMNo3nJsBXfcpJTVhZcXLW7RmTwTt4GVFE7suUBo9sS,0.05032830
...,...,...,...,...
6151,2023-11-17,SOL,So11111111111111111111111111111111111111111,58.04103650
6152,2023-11-16,SOL,So11111111111111111111111111111111111111111,64.06285823
6153,2023-11-15,SOL,So11111111111111111111111111111111111111111,60.34401745
6154,2023-11-14,SOL,So11111111111111111111111111111111111111111,54.44919224


In [41]:
tx_level_data['day'] = pd.to_datetime(tx_level_data['timestamp'].dt.strftime('%Y-%m-%d'))
tx_level_data

,timestamp,signature,type,source,tx_status,block_number,token_address,token_amount,direction,sender,receiver,counterparty,tx_fee,program_id,PRE_BALANCE,POST_BALANCE,SYMBOL,TOKEN_NAME,symbol,day
0,2025-04-06 12:12:19,6mPqrKS4AKYJeZ3JLjjubHSwNB96oiJdFX1DgeZNZEBHxMTEx9RUZYt2SyFRvpPohpXm9RRABbVb2CEHgpdTzGv,TRANSFER,SYSTEM_PROGRAM,success,331670303,So11111111111111111111111111111111111111111,0.00000000,received,5Hr7wZg7oBpVhH5nngRqzr5W7ZFUfCsfEhbziZJak7fr,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,5Hr7wZg7oBpVhH5nngRqzr5W7ZFUfCsfEhbziZJak7fr,0.00000583,11111111111111111111111111111111,0.01777252,0.01777252,SOL,Solana,NaN,2025-04-06
0,2025-04-06 00:10:50,29aP1yrCx2dbJM4Mxk8257G5e87bY7CgxySDS6LiAAmsxXLspNqdbGpYZFYiwJqjPLwBU5V6gwZ2gvuf4FSc5YrU,TRANSFER,SYSTEM_PROGRAM,success,331561835,So11111111111111111111111111111111111111111,0.00000000,received,FLiPgGTXtBtEJoytikaywvWgbz5a56DdHKZU72HSYMFF,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,FLiPgGTXtBtEJoytikaywvWgbz5a56DdHKZU72HSYMFF,0.00000500,11111111111111111111111111111111,0.01777252,0.01777252,SOL,Solana,NaN,2025-04-06
0,2025-04-06 00:10:45,2D7D4ndxhZfwZ6TRN6pzY7PQEFkRKTQhQjQ1zpz5MyNDVSP4BGV7NiSja2EvF1bKMa6NfudBP2DQjoocAibqqGQs,TRANSFER,SYSTEM_PROGRAM,success,331561822,So11111111111111111111111111111111111111111,0.00000000,received,FLiPGqowc82LLR173hKiFYBq2fCxLZEST5iHbHwj8xKb,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,FLiPGqowc82LLR173hKiFYBq2fCxLZEST5iHbHwj8xKb,0.00000500,11111111111111111111111111111111,0.01777252,0.01777252,SOL,Solana,NaN,2025-04-06
0,2025-04-05 23:38:43,nPrzWnG7SLNepcPeBWjEMKadD6WNuWScRgEwEqSNsHiZNN5kJqsniF9ArMZRStMH5T4GnNcUU97YXecsyXocsJ9,TRANSFER,SYSTEM_PROGRAM,success,331557009,So11111111111111111111111111111111111111111,0.00001000,received,6UgXZZBoydXRNX6SLjwFDUUgn19GhjBpvNAXU3iepzUV,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,6UgXZZBoydXRNX6SLjwFDUUgn19GhjBpvNAXU3iepzUV,0.00000500,11111111111111111111111111111111,0.01776252,0.01777252,SOL,Solana,NaN,2025-04-05
0,2025-04-05 23:38:25,yqSAMWuNg3pC9H5EZJmTr1MudCuUajZye13BMfaS2atLvCUPiKx3FzcxhXZEpE48dKZ7Qt2PqKmh7JRUhY4FxTZ,TRANSFER,SYSTEM_PROGRAM,success,331556964,So11111111111111111111111111111111111111111,0.00000000,received,GUq7PhyAUZko2mPhv3CupmdJKQ61LH8VyrdsRL25q7zg,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,GUq7PhyAUZko2mPhv3CupmdJKQ61LH8VyrdsRL25q7zg,0.00000500,11111111111111111111111111111111,0.01776252,0.01776252,SOL,Solana,NaN,2025-04-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,2023-11-14 15:25:34,YVCTdNEzqoFcVZV66mkasUUWK3cdz63WshXtshTUafxN52BfdS85rkpauSYgk2mEvqZW4q4miWa7G5Rw6gwpf6u,TOKEN_MINT,UNKNOWN,failed,229988531,So11111111111111111111111111111111111111111,0.00203928,sent,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,HzXh3xG4ze9Nw5iBWyUEtGHDXGg6qV2dRtesK5irZsrF,HzXh3xG4ze9Nw5iBWyUEtGHDXGg6qV2dRtesK5irZsrF,0.00000514,TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA,0.46986227,0.46985713,SOL,Solana,NaN,2023-11-14
2,2023-11-14 15:25:34,YVCTdNEzqoFcVZV66mkasUUWK3cdz63WshXtshTUafxN52BfdS85rkpauSYgk2mEvqZW4q4miWa7G5Rw6gwpf6u,TOKEN_MINT,UNKNOWN,failed,229988531,So11111111111111111111111111111111111111111,0.00203928,sent,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,EGXmJMeeXRAuTLsgk53LYXzGBawpY2Xiqq8FhMQpZauX,EGXmJMeeXRAuTLsgk53LYXzGBawpY2Xiqq8FhMQpZauX,0.00000514,TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA,0.46986227,0.46985713,SOL,Solana,NaN,2023-11-14
3,2023-11-14 15:25:34,YVCTdNEzqoFcVZV66mkasUUWK3cdz63WshXtshTUafxN52BfdS85rkpauSYgk2mEvqZW4q4miWa7G5Rw6gwpf6u,TOKEN_MINT,UNKNOWN,failed,229988531,So11111111111111111111111111111111111111111,0.00203928,received,HzXh3xG4ze9Nw5iBWyUEtGHDXGg6qV2dRtesK5irZsrF,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,HzXh3xG4ze9Nw5iBWyUEtGHDXGg6qV2dRtesK5irZsrF,0.00000514,TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA,0.46986227,0.46985713,SOL,Solana,NaN,2023-11-14
4,2023-11-14 15:25:34,YVCTdNEzqoFcVZV66mkasUUWK3cdz63WshXtshTUafxN52BfdS85rkpauSYgk2mEvqZW4q4miWa7G5Rw6gwpf6u,TOKEN_MINT,UNKNOWN,failed,229988531,So111111111111111111111

In [42]:
prices_data = prices_data.rename(columns={'dt': 'day'})


prices_data

,day,symbol,token_address,price
0,2025-04-18,BSOL,bSo13r4TkiE4KumL71LsHTPpL2euBYLFx6h9HP3piy1,162.26500000
1,2025-04-18,USDC,EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v,0.99994055
2,2025-04-18,PYUSD,2b1kV6DkPAnxd5ixfnxCpjxmKwqjjaYmCZfHsFu24GXo,0.99983073
3,2025-04-18,MSOL,mSoLzYCxHdYgdzU16g5QSh3i5K3z3KZK7ytfqcJm7So,172.44363636
4,2025-04-18,KMNO,KMNo3nJsBXfcpJTVhZcXLW7RmTwTt4GVFE7suUBo9sS,0.05032830
...,...,...,...,...
6151,2023-11-17,SOL,So11111111111111111111111111111111111111111,58.04103650
6152,2023-11-16,SOL,So11111111111111111111111111111111111111111,64.06285823
6153,2023-11-15,SOL,So11111111111111111111111111111111111111111,60.34401745
6154,2023-11-14,SOL,So11111111111111111111111111111111111111111,54.44919224


In [43]:
# Ensure datetime columns are aligned and named the same for merging
prices_data['day'] = pd.to_datetime(prices_data['day'])
prices_data['day'].min()

Timestamp('2023-11-13 00:00:00')

In [44]:
tx_level_data['signature'].nunique()

493

In [45]:
tx_level_data[tx_level_data['signature']=='4oJDpv6EPm9zyh5ifuGUSkT6qe3mNDKHLWU7sbNBwPRY8CNTwU3SdoczL65axR1RfR1gAAuC9uUWY11BFTuWTWPW']

,timestamp,signature,type,source,tx_status,block_number,token_address,token_amount,direction,sender,receiver,counterparty,tx_fee,program_id,PRE_BALANCE,POST_BALANCE,SYMBOL,TOKEN_NAME,symbol,day
0,2025-04-05 23:35:46,4oJDpv6EPm9zyh5ifuGUSkT6qe3mNDKHLWU7sbNBwPRY8CNTwU3SdoczL65axR1RfR1gAAuC9uUWY11BFTuWTWPW,UNKNOWN,JUPITER,success,331556566,Es9vMFrzaCERmJfrF4H2FYD4KCoNkY11McCe8BenwNYB,129.49945900,sent,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,39GrsozbzM9Sg1U7EDnEtQ69fsVF3pmVtmV2DGDAQQJ5,39GrsozbzM9Sg1U7EDnEtQ69fsVF3pmVtmV2DGDAQQJ5,0.00008000,JUP6LkbZbjS1jKKwapdHNy74zcZ3tLUZoi5QNyVTaV4,129.49945900,0.00000000,USDT,Tether,NaN,2025-04-05
1,2025-04-05 23:35:46,4oJDpv6EPm9zyh5ifuGUSkT6qe3mNDKHLWU7sbNBwPRY8CNTwU3SdoczL65axR1RfR1gAAuC9uUWY11BFTuWTWPW,UNKNOWN,JUPITER,success,331556566,2b1kV6DkPAnxd5ixfnxCpjxmKwqjjaYmCZfHsFu24GXo,129.41943700,received,39GrsozbzM9Sg1U7EDnEtQ69fsVF3pmVtmV2DGDAQQJ5,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,39GrsozbzM9Sg1U7EDnEtQ69fsVF3pmVtmV2DGDAQQJ5,0.00008000,JUP6LkbZbjS1jKKwapdHNy74zcZ3tLUZoi5QNyVTaV4,0.00000000,128.31937200,PYUSD,PayPal USD,NaN,2025-04-05
2,2025-04-05 23:35:46,4oJDpv6EPm9zyh5ifuGUSkT6qe3mNDKHLWU7sbNBwPRY8CNTwU3SdoczL65axR1RfR1gAAuC9uUWY11BFTuWTWPW,UNKNOWN,JUPITER,success,331556566,2b1kV6DkPAnxd5ixfnxCpjxmKwqjjaYmCZfHsFu24GXo,1.10006500,sent,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,8psNvWTrdNTiVRNzAgsou9kETXNJm2SXZyaKuJraVRtf,8psNvWTrdNTiVRNzAgsou9kETXNJm2SXZyaKuJraVRtf,0.00008000,JUP6LkbZbjS1jKKwapdHNy74zcZ3tLUZoi5QNyVTaV4,0.00000000,128.31937200,PYUSD,PayPal USD,NaN,2025-04-05


In [46]:
# Merge the price data into the tx_level_data
tx_level_data = tx_level_data.merge(prices_data[['day', 'token_address', 'price']], on=['day', 'token_address'], how='left')
tx_level_data

,timestamp,signature,type,source,tx_status,block_number,token_address,token_amount,direction,sender,receiver,counterparty,tx_fee,program_id,PRE_BALANCE,POST_BALANCE,SYMBOL,TOKEN_NAME,symbol,day,price
0,2025-04-06 12:12:19,6mPqrKS4AKYJeZ3JLjjubHSwNB96oiJdFX1DgeZNZEBHxMTEx9RUZYt2SyFRvpPohpXm9RRABbVb2CEHgpdTzGv,TRANSFER,SYSTEM_PROGRAM,success,331670303,So11111111111111111111111111111111111111111,0.00000000,received,5Hr7wZg7oBpVhH5nngRqzr5W7ZFUfCsfEhbziZJak7fr,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,5Hr7wZg7oBpVhH5nngRqzr5W7ZFUfCsfEhbziZJak7fr,0.00000583,11111111111111111111111111111111,0.01777252,0.01777252,SOL,Solana,NaN,2025-04-06,116.44750000
1,2025-04-06 00:10:50,29aP1yrCx2dbJM4Mxk8257G5e87bY7CgxySDS6LiAAmsxXLspNqdbGpYZFYiwJqjPLwBU5V6gwZ2gvuf4FSc5YrU,TRANSFER,SYSTEM_PROGRAM,success,331561835,So11111111111111111111111111111111111111111,0.00000000,received,FLiPgGTXtBtEJoytikaywvWgbz5a56DdHKZU72HSYMFF,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,FLiPgGTXtBtEJoytikaywvWgbz5a56DdHKZU72HSYMFF,0.00000500,11111111111111111111111111111111,0.01777252,0.01777252,SOL,Solana,NaN,2025-04-06,116.44750000
2,2025-04-06 00:10:45,2D7D4ndxhZfwZ6TRN6pzY7PQEFkRKTQhQjQ1zpz5MyNDVSP4BGV7NiSja2EvF1bKMa6NfudBP2DQjoocAibqqGQs,TRANSFER,SYSTEM_PROGRAM,success,331561822,So11111111111111111111111111111111111111111,0.00000000,received,FLiPGqowc82LLR173hKiFYBq2fCxLZEST5iHbHwj8xKb,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,FLiPGqowc82LLR173hKiFYBq2fCxLZEST5iHbHwj8xKb,0.00000500,11111111111111111111111111111111,0.01777252,0.01777252,SOL,Solana,NaN,2025-04-06,116.44750000
3,2025-04-05 23:38:43,nPrzWnG7SLNepcPeBWjEMKadD6WNuWScRgEwEqSNsHiZNN5kJqsniF9ArMZRStMH5T4GnNcUU97YXecsyXocsJ9,TRANSFER,SYSTEM_PROGRAM,success,331557009,So11111111111111111111111111111111111111111,0.00001000,received,6UgXZZBoydXRNX6SLjwFDUUgn19GhjBpvNAXU3iepzUV,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,6UgXZZBoydXRNX6SLjwFDUUgn19GhjBpvNAXU3iepzUV,0.00000500,11111111111111111111111111111111,0.01776252,0.01777252,SOL,Solana,NaN,2025-04-05,120.14041667
4,2025-04-05 23:38:25,yqSAMWuNg3pC9H5EZJmTr1MudCuUajZye13BMfaS2atLvCUPiKx3FzcxhXZEpE48dKZ7Qt2PqKmh7JRUhY4FxTZ,TRANSFER,SYSTEM_PROGRAM,success,331556964,So11111111111111111111111111111111111111111,0.00000000,received,GUq7PhyAUZko2mPhv3CupmdJKQ61LH8VyrdsRL25q7zg,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,GUq7PhyAUZko2mPhv3CupmdJKQ61LH8VyrdsRL25q7zg,0.00000500,11111111111111111111111111111111,0.01776252,0.01776252,SOL,Solana,NaN,2025-04-05,120.14041667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
980,2023-11-14 15:25:34,YVCTdNEzqoFcVZV66mkasUUWK3cdz63WshXtshTUafxN52BfdS85rkpauSYgk2mEvqZW4q4miWa7G5Rw6gwpf6u,TOKEN_MINT,UNKNOWN,failed,229988531,So11111111111111111111111111111111111111111,0.00203928,sent,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,HzXh3xG4ze9Nw5iBWyUEtGHDXGg6qV2dRtesK5irZsrF,HzXh3xG4ze9Nw5iBWyUEtGHDXGg6qV2dRtesK5irZsrF,0.00000514,TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA,0.46986227,0.46985713,SOL,Solana,NaN,2023-11-14,54.44919224
981,2023-11-14 15:25:34,YVCTdNEzqoFcVZV66mkasUUWK3cdz63WshXtshTUafxN52BfdS85rkpauSYgk2mEvqZW4q4miWa7G5Rw6gwpf6u,TOKEN_MINT,UNKNOWN,failed,229988531,So11111111111111111111111111111111111111111,0.00203928,sent,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,EGXmJMeeXRAuTLsgk53LYXzGBawpY2Xiqq8FhMQpZauX,EGXmJMeeXRAuTLsgk53LYXzGBawpY2Xiqq8FhMQpZauX,0.00000514,TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA,0.46986227,0.46985713,SOL,Solana,NaN,2023-11-14,54.44919224
982,2023-11-14 15:25:34,YVCTdNEzqoFcVZV66mkasUUWK3cdz63WshXtshTUafxN52BfdS85rkpauSYgk2mEvqZW4q4miWa7G5Rw6gwpf6u,TOKEN_MINT,UNKNOWN,failed,229988531,So11111111111111111111111111111111111111111,0.00203928,received,HzXh3xG4ze9Nw5iBWyUEtGHDXGg6qV2dRtesK5irZsrF,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,HzXh3xG4ze9Nw5iBWyUEtGHDXGg6qV2dRtesK5irZsrF,0.00000514,TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA,0.46986227,0.46985713,SOL,Solana,NaN,2023-11-14,54.44919224
983,2023-11-14 15:25:34,YVCTdNEzqoFcVZV66mkasUUWK3cd

In [47]:
tx_level_data[tx_level_data['signature']=='4oJDpv6EPm9zyh5ifuGUSkT6qe3mNDKHLWU7sbNBwPRY8CNTwU3SdoczL65axR1RfR1gAAuC9uUWY11BFTuWTWPW']

,timestamp,signature,type,source,tx_status,block_number,token_address,token_amount,direction,sender,receiver,counterparty,tx_fee,program_id,PRE_BALANCE,POST_BALANCE,SYMBOL,TOKEN_NAME,symbol,day,price
8,2025-04-05 23:35:46,4oJDpv6EPm9zyh5ifuGUSkT6qe3mNDKHLWU7sbNBwPRY8CNTwU3SdoczL65axR1RfR1gAAuC9uUWY11BFTuWTWPW,UNKNOWN,JUPITER,success,331556566,Es9vMFrzaCERmJfrF4H2FYD4KCoNkY11McCe8BenwNYB,129.49945900,sent,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,39GrsozbzM9Sg1U7EDnEtQ69fsVF3pmVtmV2DGDAQQJ5,39GrsozbzM9Sg1U7EDnEtQ69fsVF3pmVtmV2DGDAQQJ5,0.00008000,JUP6LkbZbjS1jKKwapdHNy74zcZ3tLUZoi5QNyVTaV4,129.49945900,0.00000000,USDT,Tether,NaN,2025-04-05,0.99964546
9,2025-04-05 23:35:46,4oJDpv6EPm9zyh5ifuGUSkT6qe3mNDKHLWU7sbNBwPRY8CNTwU3SdoczL65axR1RfR1gAAuC9uUWY11BFTuWTWPW,UNKNOWN,JUPITER,success,331556566,2b1kV6DkPAnxd5ixfnxCpjxmKwqjjaYmCZfHsFu24GXo,129.41943700,received,39GrsozbzM9Sg1U7EDnEtQ69fsVF3pmVtmV2DGDAQQJ5,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,39GrsozbzM9Sg1U7EDnEtQ69fsVF3pmVtmV2DGDAQQJ5,0.00008000,JUP6LkbZbjS1jKKwapdHNy74zcZ3tLUZoi5QNyVTaV4,0.00000000,128.31937200,PYUSD,PayPal USD,NaN,2025-04-05,0.99981887
10,2025-04-05 23:35:46,4oJDpv6EPm9zyh5ifuGUSkT6qe3mNDKHLWU7sbNBwPRY8CNTwU3SdoczL65axR1RfR1gAAuC9uUWY11BFTuWTWPW,UNKNOWN,JUPITER,success,331556566,2b1kV6DkPAnxd5ixfnxCpjxmKwqjjaYmCZfHsFu24GXo,1.10006500,sent,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,8psNvWTrdNTiVRNzAgsou9kETXNJm2SXZyaKuJraVRtf,8psNvWTrdNTiVRNzAgsou9kETXNJm2SXZyaKuJraVRtf,0.00008000,JUP6LkbZbjS1jKKwapdHNy74zcZ3tLUZoi5QNyVTaV4,0.00000000,128.31937200,PYUSD,PayPal USD,NaN,2025-04-05,0.99981887


In [48]:
# Create the token_amount_usd column
tx_level_data['token_amount_usd'] = tx_level_data['token_amount'].fillna(0) * tx_level_data['price']



In [49]:
tx_level_data[tx_level_data['token_address']=='COMPRESSED_NFT']

,timestamp,signature,type,source,tx_status,block_number,token_address,token_amount,direction,sender,receiver,counterparty,tx_fee,program_id,PRE_BALANCE,POST_BALANCE,SYMBOL,TOKEN_NAME,symbol,day,price,token_amount_usd
125,2025-02-03 17:43:37,3TPZSpTA5QKz5AQ3g7yX2ELnQHWYRGiuKu9T8GgVjBfSvZGgjxfi7eVporwQGaEWzw1JhM56Nn8sk3ijSPFd8nML,COMPRESSED_NFT_MINT,BUBBLEGUM,success,318251172,COMPRESSED_NFT,1.00000000,received,GwxDwNvmh5exDPJibZL2ztiecW7aSYoa2rca3zvN1MRB,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,GwxDwNvmh5exDPJibZL2ztiecW7aSYoa2rca3zvN1MRB,0.00000500,BGUMAp9Gq7iTEuizy4pqaxsTyUCBK68MDfK752saRPUY,NaN,NaN,NaN,NaN,AB Open #001,2025-02-03,NaN,NaN
127,2025-02-03 17:43:22,35ie9YJLNVcMGA751C2aN2JHoN7YidMmgntEw2pg3QLEzwM1qKAwdDfUxfkrjvCaDFCeqECUxnopH6v3K6SYC7Cm,COMPRESSED_NFT_MINT,BUBBLEGUM,success,318251136,COMPRESSED_NFT,1.00000000,received,GwxDwNvmh5exDPJibZL2ztiecW7aSYoa2rca3zvN1MRB,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,GwxDwNvmh5exDPJibZL2ztiecW7aSYoa2rca3zvN1MRB,0.00000500,BGUMAp9Gq7iTEuizy4pqaxsTyUCBK68MDfK752saRPUY,NaN,NaN,NaN,NaN,AB Open #001,2025-02-03,NaN,NaN
225,2024-12-31 01:27:48,5PKmHyHf9C6ASTt2HWqXy3iMuRLofE3i6HAB88kYEHUuFCRKhyDq5gyyotq3yCxvs1xZRP9CF1C8CJDQ7yPYd7yv,COMPRESSED_NFT_MINT,BUBBLEGUM,success,310883171,COMPRESSED_NFT,1.00000000,received,59suVJh2jLvADqgxv3cUmNMm9g3hjXb5M6vu29jV27Tt,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,59suVJh2jLvADqgxv3cUmNMm9g3hjXb5M6vu29jV27Tt,0.00000632,BGUMAp9Gq7iTEuizy4pqaxsTyUCBK68MDfK752saRPUY,NaN,NaN,NaN,NaN,Grant #3,2024-12-31,NaN,NaN
241,2024-12-30 17:20:28,5zMbFS9efMXYjwN28zgzDGo5j8VNiPikgc8YAv1EfHUBtX1NgsvoUjduMN5fjw89wt6YHZt6QmZcW7Weqj5b3GGU,COMPRESSED_NFT_MINT,BUBBLEGUM,success,310811606,COMPRESSED_NFT,1.00000000,received,CaUju6XDMoLWpd5if1es3FQWFR71tzGMQ9SsUSnceBYX,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,CaUju6XDMoLWpd5if1es3FQWFR71tzGMQ9SsUSnceBYX,0.00000500,BGUMAp9Gq7iTEuizy4pqaxsTyUCBK68MDfK752saRPUY,NaN,NaN,NaN,NaN,🌟 Gift #314,2024-12-30,NaN,NaN
242,2024-12-30 17:20:28,5zMbFS9efMXYjwN28zgzDGo5j8VNiPikgc8YAv1EfHUBtX1NgsvoUjduMN5fjw89wt6YHZt6QmZcW7Weqj5b3GGU,COMPRESSED_NFT_MINT,BUBBLEGUM,success,310811606,COMPRESSED_NFT,1.00000000,received,CaUju6XDMoLWpd5if1es3FQWFR71tzGMQ9SsUSnceBYX,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,CaUju6XDMoLWpd5if1es3FQWFR71tzGMQ9SsUSnceBYX,0.00000500,BGUMAp9Gq7iTEuizy4pqaxsTyUCBK68MDfK752saRPUY,NaN,NaN,NaN,NaN,🌟 Gift #320,2024-12-30,NaN,NaN
250,2024-12-29 15:35:01,2DVEywsq4Qt2KgkmZMuP3FrB1UZVPN6Rvi9rHUFYmoBEzzhrsQTRv3tyJaMt6AthVy7q1wdmrN6JDHnkqGhpBFHn,COMPRESSED_NFT_MINT,BUBBLEGUM,success,310585694,COMPRESSED_NFT,1.00000000,received,5JQ8b8ZHeu6iSKH8gH7Gee5jWPfW6hc4BZrW4FZpq3Kh,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,5JQ8b8ZHeu6iSKH8gH7Gee5jWPfW6hc4BZrW4FZpq3Kh,0.00000632,BGUMAp9Gq7iTEuizy4pqaxsTyUCBK68MDfK752saRPUY,NaN,NaN,NaN,NaN,☄️JUPITER☄️ AIRDROP,2024-12-29,NaN,NaN
252,2024-12-29 15:34:31,4Fnj33Jq5xe7MZcdVa15su8VKMXXW6Jc2y6d4FDeoVEQj5v9ffQT8HQHPBxc5mKJWrV7q5vpDfM6vMariktDmE2x,COMPRESSED_NFT_MINT,BUBBLEGUM,success,310585620,COMPRESSED_NFT,1.00000000,received,5JQ8b8ZHeu6iSKH8gH7Gee5jWPfW6hc4BZrW4FZpq3Kh,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,5JQ8b8ZHeu6iSKH8gH7Gee5jWPfW6hc4BZrW4FZpq3Kh,0.00000632,BGUMAp9Gq7iTEuizy4pqaxsTyUCBK68MDfK752saRPUY,NaN,NaN,NaN,NaN,☄️JUPITER☄️ AIRDROP,2024-12-29,NaN,NaN
257,2024-12-23 16:10:06,HB6rZmSiSNJoDTz45cwdmRDMCnJZKdLod2v9shroCBAdwfsWRZsP916CrQWqg6dpgeUeJsnouz7DZSq49M87uBN,COMPRESSED_NFT_MINT,BUBBLEGUM,success,309332138,COMPRESSED_NFT,1.00000000,received,4AuVuuzh7NA8b8jCcJzARdUALum4MxG9tPYA7QxTPHqo,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,4AuVuuzh7NA8b8jCcJzARdUALum4MxG9tPYA7QxTPHqo,0.00000730,BGUMAp9Gq7iTEuizy4pqaxsTyUCBK68MDfK752saRPUY,NaN,NaN,NaN,NaN,BOX#8,2024-12-23,NaN,NaN
275,2024-12-21 22:38:42,4gtgKHj7aum7JBcb4Bm9Norw3216dQkzHKtxBFznskoB48FYGTUdmhnP7siG3z6juKwUN7dH1sj91Lypc2bq3unr,COMPRESSED_NFT_MINT,BUBBLEGUM,success,308967649,COMPRESSED_NFT,1.00000000,received,4AuVuuzh7NA8b8jCcJzARdUALum4MxG9tPYA7QxTPHqo,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,4AuVuuzh7NA8b8jCc

In [50]:
tx_level_data[tx_level_data['signature']=='4oJDpv6EPm9zyh5ifuGUSkT6qe3mNDKHLWU7sbNBwPRY8CNTwU3SdoczL65axR1RfR1gAAuC9uUWY11BFTuWTWPW']

,timestamp,signature,type,source,tx_status,block_number,token_address,token_amount,direction,sender,receiver,counterparty,tx_fee,program_id,PRE_BALANCE,POST_BALANCE,SYMBOL,TOKEN_NAME,symbol,day,price,token_amount_usd
8,2025-04-05 23:35:46,4oJDpv6EPm9zyh5ifuGUSkT6qe3mNDKHLWU7sbNBwPRY8CNTwU3SdoczL65axR1RfR1gAAuC9uUWY11BFTuWTWPW,UNKNOWN,JUPITER,success,331556566,Es9vMFrzaCERmJfrF4H2FYD4KCoNkY11McCe8BenwNYB,129.49945900,sent,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,39GrsozbzM9Sg1U7EDnEtQ69fsVF3pmVtmV2DGDAQQJ5,39GrsozbzM9Sg1U7EDnEtQ69fsVF3pmVtmV2DGDAQQJ5,0.00008000,JUP6LkbZbjS1jKKwapdHNy74zcZ3tLUZoi5QNyVTaV4,129.49945900,0.00000000,USDT,Tether,NaN,2025-04-05,0.99964546,129.45354604
9,2025-04-05 23:35:46,4oJDpv6EPm9zyh5ifuGUSkT6qe3mNDKHLWU7sbNBwPRY8CNTwU3SdoczL65axR1RfR1gAAuC9uUWY11BFTuWTWPW,UNKNOWN,JUPITER,success,331556566,2b1kV6DkPAnxd5ixfnxCpjxmKwqjjaYmCZfHsFu24GXo,129.41943700,received,39GrsozbzM9Sg1U7EDnEtQ69fsVF3pmVtmV2DGDAQQJ5,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,39GrsozbzM9Sg1U7EDnEtQ69fsVF3pmVtmV2DGDAQQJ5,0.00008000,JUP6LkbZbjS1jKKwapdHNy74zcZ3tLUZoi5QNyVTaV4,0.00000000,128.31937200,PYUSD,PayPal USD,NaN,2025-04-05,0.99981887,129.39599590
10,2025-04-05 23:35:46,4oJDpv6EPm9zyh5ifuGUSkT6qe3mNDKHLWU7sbNBwPRY8CNTwU3SdoczL65axR1RfR1gAAuC9uUWY11BFTuWTWPW,UNKNOWN,JUPITER,success,331556566,2b1kV6DkPAnxd5ixfnxCpjxmKwqjjaYmCZfHsFu24GXo,1.10006500,sent,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,8psNvWTrdNTiVRNzAgsou9kETXNJm2SXZyaKuJraVRtf,8psNvWTrdNTiVRNzAgsou9kETXNJm2SXZyaKuJraVRtf,0.00008000,JUP6LkbZbjS1jKKwapdHNy74zcZ3tLUZoi5QNyVTaV4,0.00000000,128.31937200,PYUSD,PayPal USD,NaN,2025-04-05,0.99981887,1.09986575


In [51]:
balance_df[balance_df['TX_ID']=='2iqhT33xr4mhcRKvRTTZKqGCw16m8USVVV2XHXm2ZGCgCHS63zeDuFWP2HztKvh2Tj5tpemKBGEEnjUQYK5Umvj3']

,BLOCK_TIMESTAMP,OWNER,MINT,PRE_BALANCE,BALANCE,TX_ID,SUCCEEDED,SYMBOL,NAME
611,2023-11-14 15:43:23.000,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,So11111111111111111111111111111111111111111,0.46985713,0.23288380,2iqhT33xr4mhcRKvRTTZKqGCw16m8USVVV2XHXm2ZGCgCHS63zeDuFWP2HztKvh2Tj5tpemKBGEEnjUQYK5Umvj3,True,SOL,Solana
612,2023-11-14 15:43:23.000,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v,0.00000000,13.16214900,2iqhT33xr4mhcRKvRTTZKqGCw16m8USVVV2XHXm2ZGCgCHS63zeDuFWP2HztKvh2Tj5tpemKBGEEnjUQYK5Umvj3,True,USDC,USDC


In [52]:
tx_level_data[tx_level_data['type']=='SWAP']

,timestamp,signature,type,source,tx_status,block_number,token_address,token_amount,direction,sender,receiver,counterparty,tx_fee,program_id,PRE_BALANCE,POST_BALANCE,SYMBOL,TOKEN_NAME,symbol,day,price,token_amount_usd
142,2025-02-03 17:33:00,5fpGxwNLgfecyzak9LKuXQtYrYHGv37EtzzHBTVciCQMQSSBwoW3wJMETbgZkvWdxTg5p9736TJH35SGWG3Upers,SWAP,JUPITER,success,318249572,So11111111111111111111111111111111111111111,0.00203928,sent,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,HzXh3xG4ze9Nw5iBWyUEtGHDXGg6qV2dRtesK5irZsrF,HzXh3xG4ze9Nw5iBWyUEtGHDXGg6qV2dRtesK5irZsrF,0.00080500,TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA,0.00312518,0.02621129,SOL,Solana,NaN,2025-02-03,200.11638550,0.40809334
143,2025-02-03 17:33:00,5fpGxwNLgfecyzak9LKuXQtYrYHGv37EtzzHBTVciCQMQSSBwoW3wJMETbgZkvWdxTg5p9736TJH35SGWG3Upers,SWAP,JUPITER,success,318249572,So11111111111111111111111111111111111111111,0.02593039,received,HzXh3xG4ze9Nw5iBWyUEtGHDXGg6qV2dRtesK5irZsrF,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,HzXh3xG4ze9Nw5iBWyUEtGHDXGg6qV2dRtesK5irZsrF,0.00080500,TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA,0.00312518,0.02621129,SOL,Solana,NaN,2025-02-03,200.11638550,5.18909692
144,2025-02-03 17:33:00,5fpGxwNLgfecyzak9LKuXQtYrYHGv37EtzzHBTVciCQMQSSBwoW3wJMETbgZkvWdxTg5p9736TJH35SGWG3Upers,SWAP,JUPITER,success,318249572,Es9vMFrzaCERmJfrF4H2FYD4KCoNkY11McCe8BenwNYB,5.00000000,sent,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,4xDsmeTWPNjgSVSS1VTfzFq3iHZhp77ffPkAmkZkdu71,4xDsmeTWPNjgSVSS1VTfzFq3iHZhp77ffPkAmkZkdu71,0.00080500,TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA,497.86208500,492.86208500,USDT,Tether,NaN,2025-02-03,1.00077204,5.00386021
145,2025-02-03 17:33:00,5fpGxwNLgfecyzak9LKuXQtYrYHGv37EtzzHBTVciCQMQSSBwoW3wJMETbgZkvWdxTg5p9736TJH35SGWG3Upers,SWAP,JUPITER,success,318249572,So11111111111111111111111111111111111111112,0.02389112,received,4xDsmeTWPNjgSVSS1VTfzFq3iHZhp77ffPkAmkZkdu71,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,4xDsmeTWPNjgSVSS1VTfzFq3iHZhp77ffPkAmkZkdu71,0.00080500,TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA,NaN,NaN,NaN,NaN,NaN,2025-02-03,200.11638550,4.78100358
332,2024-12-14 18:25:32,5btVurEw7jG6SRr4Np6npL2r1Q7dwmGrJXy79ofhz5pfrdXzAe5Ebzjy9usFY5jbZ95cixsVpwDRNzM5cz3zwTsG,SWAP,JUPITER,success,307474693,So11111111111111111111111111111111111111111,1.11000000,sent,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,HzXh3xG4ze9Nw5iBWyUEtGHDXGg6qV2dRtesK5irZsrF,HzXh3xG4ze9Nw5iBWyUEtGHDXGg6qV2dRtesK5irZsrF,0.00040500,TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA,1.13076693,0.02036193,SOL,Solana,NaN,2024-12-14,223.10708333,247.64886250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974,2023-11-14 15:43:23,2iqhT33xr4mhcRKvRTTZKqGCw16m8USVVV2XHXm2ZGCgCHS63zeDuFWP2HztKvh2Tj5tpemKBGEEnjUQYK5Umvj3,SWAP,JUPITER,success,229990897,So11111111111111111111111111111111111111111,0.00203928,sent,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,HzXh3xG4ze9Nw5iBWyUEtGHDXGg6qV2dRtesK5irZsrF,HzXh3xG4ze9Nw5iBWyUEtGHDXGg6qV2dRtesK5irZsrF,0.00000549,TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA,0.46985713,0.23288380,SOL,Solana,NaN,2023-11-14,54.44919224,0.11103715
975,2023-11-14 15:43:23,2iqhT33xr4mhcRKvRTTZKqGCw16m8USVVV2XHXm2ZGCgCHS63zeDuFWP2HztKvh2Tj5tpemKBGEEnjUQYK5Umvj3,SWAP,JUPITER,success,229990897,So11111111111111111111111111111111111111111,0.00203928,sent,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,EGXmJMeeXRAuTLsgk53LYXzGBawpY2Xiqq8FhMQpZauX,EGXmJMeeXRAuTLsgk53LYXzGBawpY2Xiqq8FhMQpZauX,0.00000549,TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA,0.46985713,0.23288380,SOL,Solana,NaN,2023-11-14,54.44919224,0.11103715
976,2023-11-14 15:43:23,2iqhT33xr4mhcRKvRTTZKqGCw16m8USVVV2XHXm2ZGCgCHS63zeDuFWP2HztKvh2Tj5tpemKBGEEnjUQYK5Umvj3,SWAP,JUPITER,success,229990897,So11111111111111111111111111111111111111111,0.00203928,received,HzXh3xG4ze9Nw5iBWyUEtGHDXGg6qV2dRtesK5irZsrF,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,HzXh3xG4ze9Nw5iBWyUEtGHDXGg6qV2dRtesK5irZsrF,0.00000549,TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA,0.46985713,0.23288380,SOL,Solana,NaN

In [62]:
tx_level_data.columns

Index(['timestamp', 'signature', 'type', 'source', 'tx_status', 'block_number', 'token_address', 'token_amount', 'direction', 'sender', 'receiver', 'counterparty', 'tx_fee', 'program_id',
       'PRE_BALANCE', 'POST_BALANCE', 'SYMBOL', 'TOKEN_NAME', 'symbol', 'day', 'price', 'token_amount_usd'],
      dtype='object')

In [64]:
#Confirming datatypes 

float_cols = ['token_amount_usd','token_amount', 'tx_fee','PRE_BALANCE', 'POST_BALANCE']
int_cols = ['block_number']
str_cols = ['signature',
       'sender', 'receiver', 'counterparty','SYMBOL', 'TOKEN_NAME', 'token_address']
category_cols = ['type','source','tx_status','direction']

tx_level_data[float_cols] = tx_level_data[float_cols].astype(float)
tx_level_data[int_cols] = tx_level_data[int_cols].astype(float)
tx_level_data[str_cols] = tx_level_data[str_cols].astype(str)
tx_level_data[category_cols] = tx_level_data[category_cols].astype("category")

# Saving the Dataset

In [65]:
tx_level_data.to_csv('../data/processed/test_acc_timeseries.csv', index=False)

In [66]:
tx_level_data

%store tx_level_data

Stored 'tx_level_data' (DataFrame)


# Nansen Data

In [56]:
# I think the API is not public yet

# Misc

In [57]:
# # helpers/utility.py
# def _make_post_request(url: str, payload: dict, headers: dict) -> dict:
#     response = requests.post(url, json=payload, headers=headers)
#     if response.status_code == 200:
#         return response.json()
#     else:
#         raise ValueError(f"Error: {response.status_code}: {response.text}")

In [58]:
# # Initialize with your API key (ensure it's valid and active)
# #HELIUS_API_KEY = os.getenv("api_key")  
# tx_api = TransactionsAPI(api_key=HELIUS_API_KEY)

# tx_api

In [59]:
# # Example usage
# transactions = [""]  # Your transaction signatures
# response = tx_api.get_parsed_transactions(transactions)
# print(response)

In [60]:
balances_api = BalancesAPI(HELIUS_API_KEY)
balances_api

In [61]:
# balances_api.get_balances("hq8fC9VkxU6rv1QvMUkGKbhFR3MxSd7nCkCGaaXx9MA")